# method 1

In [1]:
import pandas as pd
import tTEM_toolbox as tt
from pathlib import Path
#from plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import plotly.io as pio
pio.kaleido.scope.chromium_args = tuple([arg for arg in pio.kaleido.scope.chromium_args if arg != "--disable-dev-shm-usage"])
workdir = Path(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test')
welllog = workdir.joinpath(r'Plot_with_well_log\Well_log.xlsx')
ttemname_north = workdir.joinpath(r'Plot_with_well_log\PD1_I01_MOD.xyz')
ttemname_center = workdir.joinpath(r'Plot_with_well_log\PD22_I03_MOD.xyz')
DOI = workdir.joinpath(r'Plot_with_well_log\DOID1_DOIStaE.xyz')

In [2]:
ttem_north = tt.main.ProcessTTEM(ttem_path=[ttemname_north],
                                  welllog=welllog,
                                  DOI_path=DOI)
ttem_north_data = ttem_north.data()
matched_ttem, matched_well = tt.bootstrap.select_closest(ttem_north_data,welllog)

Applying DOI |                                | 77/7968

No layer was excluded
No line was excluded
No point was excluded


Applying DOI |################################| 7968/7968
Formating well logs |################################| 223/223
Selecting closest ttem trace |################################| 223/223

Total of 7 well with in radius (500m), 216 skipped


In [16]:
ttem_center = tt.main.ProcessTTEM(ttem_path=[ttemname_center],
                                  welllog=welllog,
                                  DOI_path=DOI)
ttem_center_data = ttem_center.data()
matched_ttem, matched_well = tt.bootstrap.select_closest(ttem_center_data,welllog)

Applying DOI |                                | 87/7968

No layer was excluded
No line was excluded
No point was excluded


Applying DOI |################################| 7968/7968
Formating well logs |################################| 223/223
Selecting closest ttem trace |################################| 223/223


Total of 40 well with in radius (500m), 183 skipped


In [15]:
well_grouped = matched_well.groupby('Bore')
ttem_group = matched_ttem.groupby('Bore')
concatlist = []
for name, group in well_grouped:
    ttem_DOI_elevation_min = ttem_group.get_group(name).Elevation_End.min()
    well_depth_to_ttem_DOI = group[group['Elevation_End']>= ttem_DOI_elevation_min]
    total_thickness = well_depth_to_ttem_DOI['Thickness'].sum()
    lithology = well_depth_to_ttem_DOI.groupby('Keyword')
    try:
        well_fine_grain_pct = lithology.get_group('fine grain')['Thickness'].sum()/total_thickness
    except KeyError:
        well_fine_grain_pct = 0
    try:
        well_mix_grain_pct = lithology.get_group('mix grain')['Thickness'].sum()/total_thickness
    except KeyError:
        well_mix_grain_pct = 0
    try:
        well_coarse_grain_pct = lithology.get_group('coarse grain')['Thickness'].sum()/total_thickness
    except KeyError:
        well_coarse_grain_pct = 0
    for rho_fine in range(1,20):
        for rho_coarse in range(1,50):
            rock_resistivity_range_df = pd.DataFrame({'Fine_conf':[0,rho_fine],'Mix_conf':[16,19],'Coarse_conf':[rho_coarse,200]})
            ttem_indiv_group = ttem_group.get_group(name)
            rock_trans = tt.Rock_trans.rock_transform(ttem_indiv_group,rock_resistivity_range_df)
            group_total_thickness = rock_trans['Thickness'].sum()
            rock_trans_grain_types = rock_trans.groupby('Identity')
            try:
                ttem_fine_grain_pct = rock_trans_grain_types.get_group('Fine_grain')['Thickness'].sum()/group_total_thickness
            except KeyError:
                ttem_fine_grain_pct = 0
            try:
                ttem_mix_grain_pct = rock_trans_grain_types.get_group('Mix_grain')['Thickness'].sum()/group_total_thickness
            except KeyError:
                ttem_mix_grain_pct = 0
            try:
                ttem_coarse_grain_pct = rock_trans_grain_types.get_group('Coarse_grain')['Thickness'].sum()/group_total_thickness
            except KeyError:
                ttem_coarse_grain_pct = 0
            tmp_df =  pd.DataFrame([dict(Bore=name,rho_fine=rho_fine,rho_coarse=rho_coarse, fine_grain_error = abs(ttem_fine_grain_pct-well_fine_grain_pct), mix_grain_error = abs(ttem_mix_grain_pct-well_mix_grain_pct), coarse_grain_error = abs(ttem_coarse_grain_pct-well_coarse_grain_pct))])
            concatlist.append(tmp_df)
lithology_pct_error = pd.concat(concatlist)
lithology_pct_error['Total_error'] = lithology_pct_error['fine_grain_error'] + lithology_pct_error['mix_grain_error']+lithology_pct_error['coarse_grain_error']
lithology_pct_error_group = lithology_pct_error.groupby('Bore')

In [11]:
lithology_pct_error_group

In [16]:
for name, group in lithology_pct_error_group:
    fig = []
    fig = px.scatter(group,x='fine_grain_error',y='coarse_grain_error', custom_data=['rho_fine','rho_coarse'])
    fig.update_traces(hovertemplate =
                        '<b>fine_grain_error: %{x}</b><br>'+
                        '<b>coarse_grain_error: %{y}</b><br>'+
                      '<b>rho_fine: %{customdata[0]}</b><br>'+
                      '<b>rho_coarse: %{customdata[1]}</b><br>')
    fig.update_layout(title = name)
    import time
    time.sleep(2)
    fig.show()

In [18]:
for name, group in lithology_pct_error_group:
    fig = []
    fig = px.scatter(group,x='rho_fine',y='rho_coarse', custom_data=['fine_grain_error','mix_grain_error','coarse_grain_error','Total_error'], color='Total_error',color_continuous_scale='RdBu_r',)
    fig.update_traces(hovertemplate =
                        '<b>rho_fine: %{x}</b><br>'+
                        '<b>rho_coarse: %{y}</b><br>'+
                      '<b>fine_grain_error: %{customdata[0]:.2f}</b><br>'+
                      '<b>mix_grain_error: %{customdata[1]:.2f}</b><br>'+
                      '<b>coarse_grain_error: %{customdata[2]:.2f}</b><br>'+
                      '<b>Total_error: %{customdata[3]:.2f}</b><br>')
    fig.update_layout(title = name)
    fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')
    fig.update_traces(marker=dict(size=12))
    import time
    time.sleep(2)
    fig.show()

In [19]:
lithology_pct_error_group.groups.keys()

dict_keys([' 1788', ' 5327', ' 5450', '1234999', '13909', '13911', '13916', '13918', '13924', '13926', '13959', '13982', '13983', '13984', '13991', '13992', '14648', '17425', '20763', '23280', '23655', '25423', '26425', '26426', '26820', '26908', '27443', '28966', '30092', '34452', '426887', '430589', '431341', '6252', '6562', '7090', '7766', '8560', '9173', '9447'])

In [5]:
import os
os.startfile(workdir)

In [14]:
concatlist2 = []
for name, group in ttem_group:
    for rho_fine in range(1,20):
        for rho_coarse in range(1,50):
            rock_resistivity_range_df = pd.DataFrame({'Fine_conf':[0,rho_fine],'Mix_conf':[16,19],'Coarse_conf':[rho_coarse,200]})
            rock_trans = tt.Rock_trans.rock_transform(group,rock_resistivity_range_df)
            group_total_thickness = rock_trans['Thickness'].sum()
            rock_trans_grain_types = rock_trans.groupby('Identity')
            try:
                fine_grain_pct = rock_trans_grain_types.get_group('Fine_grain')['Thickness'].sum()/group_total_thickness
            except KeyError:
                fine_grain_pct = 0
            try:
                mix_grain_pct = rock_trans_grain_types.get_group('Mix_grain')['Thickness'].sum()/group_total_thickness
            except KeyError:
                mix_grain_pct = 0
            try:
                coarse_grain_pct = rock_trans_grain_types.get_group('Coarse_grain')['Thickness'].sum()/group_total_thickness
            except KeyError:
                coarse_grain_pct = 0
            tmp_df = pd.DataFrame([dict(Bore=name,rho_fine=rho_fine,rho_coarse=rho_coarse,fine_grain = fine_grain_pct, mix_grain = mix_grain_pct, coarse_grain = coarse_grain_pct)])
            concatlist2.append(tmp_df)
rk_transform_lithology_pct = pd.concat(concatlist2)




In [9]:
rk_transform_lithology_pct

,Bore,rho_fine,rho_coarse,fine_grain,mix_grain,coarse_grain
0,0375002P00 27210,5,21,0.0,0.627536,0.372464
0,0375002P00 27210,5,22,0.0,0.627536,0.372464
0,0375002P00 27210,5,23,0.0,0.711422,0.288578
0,0375002P00 27210,5,24,0.0,0.859365,0.140635
0,0375002P00 27210,5,25,0.0,0.958307,0.041693
...,...,...,...,...,...,...
0,7344,19,45,0.0,0.437758,0.562242
0,7344,19,46,0.0,0.460599,0.539401
0,7344,19,47,0.0,0.460599,0.539401
0,7344,19,48,0.0,0.460599,0.539401


In [7]:
result_group = rk_transform_lithology_pct.groupby('Bore')


In [8]:
result_group


In [ ]:

def method1(matched_ttem, matched_well):
    for

In [ ]:
export